# Payoff d'un iron condor sur SPY

TP sur l'iron condor (put + call spread crédit) : long put bas, short put intérieur, short call intérieur, long call haut.
Objectifs :
1. Fixer un spot de référence via l'historique SPY.
2. Visualiser le plateau central de gain limité et les pertes bornées sur les ailes.
3. Relier la structure crédit (prime reçue) à la forme du payoff.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from pathlib import Path

plt.style.use("seaborn-v0_8-darkgrid")

def fetch_spy_history(period="1y", interval="1d") -> pd.Series:
    """Récupère les prix SPY (cache local puis fallback yfinance)."""
    cache_path = Path("notebooks/GPT/_cache_spy_close.csv")
    if cache_path.exists():
        try:
            cached = pd.read_csv(cache_path, index_col=0, parse_dates=True)
            if not cached.empty and "Close" in cached.columns:
                return cached["Close"]
        except Exception:
            pass
    data = yf.download("SPY", period=period, interval=interval, progress=False)
    if data.empty or "Close" not in data:
        raise RuntimeError("Impossible de récupérer les prix SPY")
    close = data["Close"]
    try:
        cache_path.parent.mkdir(parents=True, exist_ok=True)
        close.to_csv(cache_path, index_label="date")
    except Exception:
        pass
    return close

# Récupération des prix de clôture (dernier close pour spot_ref)
close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])



def payoff_call(spot: float, strike: float) -> float:
    return max(spot - strike, 0.0)

def payoff_put(spot: float, strike: float) -> float:
    return max(strike - spot, 0.0)

def payoff_iron_condor(spot: float, k_put_long: float, k_put_short: float, k_call_short: float, k_call_long: float) -> float:
    # long put, short put, short call, long call
    return (
        payoff_put(spot, k_put_long)
        - payoff_put(spot, k_put_short)
        - payoff_call(spot, k_call_short)
        + payoff_call(spot, k_call_long)
    )

    data = yf.download("SPY", period=period, interval=interval, progress=False)
    if data.empty or "Close" not in data:
        raise RuntimeError("Impossible de récupérer les prix SPY")
    return data["Close"]

k_put_long = spot_ref * 0.9
k_put_short = spot_ref * 0.97
k_call_short = spot_ref * 1.03
k_call_long = spot_ref * 1.1


## Évolution du sous-jacent (SPY)
Clôtures sur un an et repère du spot de référence (dernier close).

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
close_spy.plot(ax=ax, color="steelblue", label="SPY close")
ax.axhline(spot_ref, color="crimson", linestyle="--", label=f"Dernier close ≈ {spot_ref:.2f}")
ax.set_title("SPY - clôtures sur 1 an")
ax.set_xlabel("Date")
ax.set_ylabel("Prix")
ax.legend()
plt.show()

## Payoff de l'iron condor
Plateau central de gain, pertes bornées sur les ailes.

In [ ]:
S_grid = np.linspace(spot_ref * 0.5, spot_ref * 1.5, 200)
payoffs = [payoff_iron_condor(S, k_put_long, k_put_short, k_call_short, k_call_long) for S in S_grid]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(S_grid, payoffs, color="navy", label="Payoff iron condor")
for k, style, label in [
    (k_put_long, ":", f"Put long = {k_put_long:.2f}"),
    (k_put_short, "--", f"Put short = {k_put_short:.2f}"),
    (k_call_short, "--", f"Call short = {k_call_short:.2f}"),
    (k_call_long, ":", f"Call long = {k_call_long:.2f}"),
]:
    ax.axvline(k, color="gray", linestyle=style, label=label)
ax.set_xlabel("Spot S")
ax.set_ylabel("Payoff à maturité")
ax.set_title("Iron condor sur SPY : payoff vs spot")
ax.legend()
plt.show()

In [ ]:

import ipywidgets as widgets
from IPython.display import display, Markdown

spot_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='Spot')
slider_k1 = widgets.FloatSlider(value=spot_ref*0.9, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K put long')
slider_k2 = widgets.FloatSlider(value=spot_ref*0.95, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K put short')
slider_k3 = widgets.FloatSlider(value=spot_ref*1.05, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K call short')
slider_k4 = widgets.FloatSlider(value=spot_ref*1.1, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K call long')
output = widgets.Output()

def _update_payoff(change=None):
    with output:
        output.clear_output()
        s = spot_slider.value
        k1 = slider_k1.value
        k2 = slider_k2.value
        k3 = slider_k3.value
        k4 = slider_k4.value
        p = payoff_iron_condor(s, k1, k2, k3, k4)
        display(Markdown(f"**Spot = {s:.2f}**

- Put long K1 = {k1:.2f}
- Put short K2 = {k2:.2f}
- Call short K3 = {k3:.2f}
- Call long K4 = {k4:.2f}
- Payoff iron condor = {p:.4f}"))

for sl in (spot_slider, slider_k1, slider_k2, slider_k3, slider_k4):
    sl.observe(_update_payoff, names='value')

_update_payoff()
display(widgets.VBox([spot_slider, slider_k1, slider_k2, slider_k3, slider_k4, output]))
